In [17]:
import json
import pandas as pd
from pyspark import SparkContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover, ChiSqSelector, IDF
from pyspark.sql import SparkSession


In [18]:
file = open("./reviews_devset.json")

reviews_devset = file.readlines()

reviewsDF = pd.DataFrame.from_records(list(map(json.loads,reviews_devset)))

print(reviewsDF)

reviewsDF["category"] = pd.Categorical(reviewsDF["category"])
reviewsDF["labels"] = reviewsDF["category"].cat.codes
print(reviewsDF.loc[:,["reviewText","category","labels"]])

           reviewerID        asin                    reviewerName helpful  \
0      A2VNYWOPJ13AFP  0981850006    Amazon Customer "carringt0n"  [6, 7]   
1      A2E5XXXC07AGA7  B00002N66D                           James  [1, 1]   
2      A16PX63WZIEQ13  B00002N67U                         Finaldx  [0, 1]   
3      A2OSWM3522VARA  B00002N6AN  Wayne Allen "Motoring Patriot"  [0, 0]   
4      A2SX9YPPGEUADI  B00002N8K3   HappyCamper "Happy Housewife"  [4, 5]   
...               ...         ...                             ...     ...   
78824  A28E5PHOPLIOVI  B00KA4H00E                             JDG  [0, 0]   
78825  A3LL2X96LT6HS6  B00KDOVQ74                  Theresa Franks  [0, 0]   
78826  A1B4BFGW8ELS7N  B00KK7K6XY                       P. Foster  [0, 0]   
78827  A2Z7B7VOR4F9ID  B00KQYEJXO                          Kramer  [0, 0]   
78828  A11FQMMFNJ1AW6  B00L1H4I0Y                      Adam Crain  [0, 0]   

                                              reviewText  overall  \
0     

In [46]:
spark = SparkSession.builder.master("local[1]").appName("part2ex2").getOrCreate()

spark_dff = spark.createDataFrame(reviewsDF.loc[:,["reviewText","category","labels"]])

/usr/lib/spark/python/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [47]:
tokenizer = Tokenizer(inputCol="reviewText", outputCol="rawWords")

remover = StopWordsRemover(
    inputCol=tokenizer.getOutputCol(),
    outputCol="words",
    caseSensitive=False)

hashingTF = HashingTF(
    inputCol=remover.getOutputCol(), outputCol="rawFeatures", numFeatures=2000)

idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")

selector = ChiSqSelector(
    numTopFeatures=2000, 
    featuresCol=idf.getOutputCol(),
    outputCol="selectedFeatures", 
    labelCol="labels"
    )

pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, selector])

model = pipeline.fit(spark_dff)

23/05/18 23:44:43 WARN TaskSetManager: Stage 27 contains a task of very large size (20042 KiB). The maximum recommended task size is 1000 KiB.
23/05/18 23:44:50 WARN TaskSetManager: Stage 28 contains a task of very large size (20042 KiB). The maximum recommended task size is 1000 KiB.
23/05/18 23:44:54 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 28 (TID 53): Attempting to kill Python Worker
23/05/18 23:44:54 WARN TaskSetManager: Stage 29 contains a task of very large size (20042 KiB). The maximum recommended task size is 1000 KiB.
23/05/18 23:45:01 WARN TaskSetManager: Stage 31 contains a task of very large size (20042 KiB). The maximum recommended task size is 1000 KiB.


In [48]:
out = model.transform(spark_dff)

In [49]:
out.toPandas().to_csv("test.csv")

23/05/18 23:45:10 WARN TaskSetManager: Stage 33 contains a task of very large size (20042 KiB). The maximum recommended task size is 1000 KiB.
